### 마켓 데이터 크롤링
# KRX 투자자별 - 구축

<img src="http://i.imgur.com/grQHNKG.jpg">

#### 2017 FinanceData http://fb.com/financedata

# 투자자별
* 시장정보 → 주식 → 순위정보 → 투자자별
* http://marketdata.krx.co.kr/contents/MKD/04/0404/04040400/MKD04040400.jsp
* 종목코드
종목명
거래량(주)
거래대금(원)
업종명
매수
매도
순매수
매수
매도
순매수

* 1999년 1월 부터 현재까지 일자별

# 투자주체의 종류

투자주체의 종류: 개인, 기관, 외국인, 기타법인 4개로 구분


### 1) 개인
* 법인에 소속되지 않은 투자자

### 2) 기관
1. 금융투자: 금융투자업자(증권회사, 선물회사, 투자자문회사)가 운용하는 자산
2. 투신: 투자자문회사를 제외한 투자신탁회사
3. 기타금융: 종합금융회사, 저축은행(신용금고), 기타법인(증권금융, 자금중개, 여신전문금융, 새마을금고, 신협 등)
4. 연기금: 연금, 기금
5. 국가/지자체: 국가, 지자체국제기구, 기타 비금융 공공기관(한국은행, 예금보험공사, 한국자산관리공사, 한국주택금융공사 등)

### 3) 외국인
1. 외국인: 금감원에 외국인투자등록되어 있거나 국내 거주기간이 6개월이내인 경우 (대부분 기관)
2. 기타외국인 : 
    * 외국인투자등록되어 있지 않은 외국인
    * 국내법인이 조세회피구역에 투자회사를 세우고 투자하는 경우

### 4) 기타법인
* 금융기관과 공공기관을 제외한 나머지 기관

# 투자주체 매매동향 크롤링

In [1]:
import io
import requests
import pandas as pd

'''
# var_invr_cd
* 1000 금융투자
* 2000 보험
* 3000 투신
* 3100 사모
* 4000 은행
* 5000 기타금융
* 6000 연기금
* 7000 국가.지자체
* 7050 기관합계
* 7100 기타법인
* 8000 개인
* 9000 외국인
* 9001 기타외국인
* 9999 전체
'''

# STEP 01: Generate OTP
gen_otp_url = 'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx?' \
    'name=fileDown&filetype=xls' \
    '&url=MKD/04/0404/04040400/mkd04040400' \
    '&stctype=ALL&var_invr_cd=%s&schdate=%s' \
    '&etctype=ST&etctype=EF&etctype=EW&etctype=EN' \
    '&pagePath=/contents/MKD/04/0404/04040400/MKD0404040'

# 9000=외국인, 20181204=날짜
r = requests.get(gen_otp_url % ('9000', '20181204'))
code = r.content

# STEP 02: download
down_url = 'http://file.krx.co.kr/download.jspx'
down_data = {'code': code}
headers = {'Referer': 'http://marketdata.krx.co.kr'}
r = requests.post(down_url, down_data, headers=headers)

df = pd.read_excel(io.BytesIO(r.content), thousands=',')
df.head()

,종목코드,종목명,매수거래량,매도거래량,순매수거래량,매수거래대금,매도거래대금,순매수거래대금,업종명
0,253450,스튜디오드래곤,438580,9032,429548,45160482700,987559500,44172923200,"영화, 비디오물, 방송프로그램 제작 및 배급업"
1,068270,셀트리온,248472,99417,149055,60913882500,24362953500,36550929000,기초 의약물질 및 생물학적 제제 제조업
2,105560,KB금융,1037057,718046,319011,49450308900,34240541650,15209767250,기타 금융업
3,055550,신한지주,714754,387205,327549,30077346950,16292433400,13784913550,기타 금융업
4,006400,삼성SDI,257825,196443,61382,56596971500,43103506000,13493465500,일차전지 및 축전지 제조업


In [2]:
len(df)

5561

# 함수로 정리 krx_investors()

'1000'(금융투자)~'9999'(전체), 14개 주체를 개별 요청하여 취합
        

In [3]:
import io
import requests
import pandas as pd
import numpy as np
from datetime import datetime

def krx_investors(dt=None):
    dt = datetime.today().date() if dt == None else pd.to_datetime(dt)
    
    # var_invr_cd 투자자 코드
    var_invr_cd = [
        ('1000', '금융투자'), 
        ('2000', '보험'), 
        ('3000', '투신'), 
        ('3100', '사모'), 
        ('4000', '은행'), 
        ('5000', '기타금융'), 
        ('6000', '연기금'), 
        ('7000', '국가.지자체'), 
        ('7050', '기관합계'), 
        ('7100', '기타법인'), 
        ('8000', '개인'), 
        ('9000', '외국인'), 
        ('9001', '기타외국인'),
        ('9999', '전체'),
    ]

    gen_otp_url = 'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx?' \
        'name=fileDown&filetype=xls' \
        '&url=MKD/04/0404/04040400/mkd04040400' \
        '&stctype=ALL&var_invr_cd=%s&schdate=%s' \
        '&etctype=ST&etctype=EF&etctype=EW&etctype=EN' \
        '&pagePath=/contents/MKD/04/0404/04040400/MKD0404040'

    df_list = []
    for invr_cd in var_invr_cd:
        # STEP 01: Generate OTP
        r = requests.get(gen_otp_url % (invr_cd[0], dt.strftime('%Y%m%d')))
        code = r.content

        # STEP 02: download
        down_url = 'http://file.krx.co.kr/download.jspx'
        down_data = {'code': code}
        headers = {'Referer': 'http://marketdata.krx.co.kr'}
        r = requests.post(down_url, down_data, headers=headers)

        df = pd.read_excel(io.BytesIO(r.content), thousands=',')
        df['투자자코드'] = invr_cd[0]
        df['투자자명'] = invr_cd[1]
        df_list.append(df)
        
    df_merged = pd.concat(df_list)
    df_merged['날짜'] = dt
    return df_merged


In [4]:
df = krx_investors('2018-10-01')
print('rows:', len(df))

df.head(10)

rows: 74844


,종목코드,종목명,매수거래량,매도거래량,순매수거래량,매수거래대금,매도거래대금,순매수거래대금,업종명,투자자코드,투자자명,날짜
0,251340,KODEX 코스닥150선물인버스,2813917,680037,2133880,17469487710,4219630655,13249857055,-,1000,금융투자,2018-10-01
1,028260,삼성물산,93885,25697,68188,12029062500,3272630000,8756432500,기타 전문 도매업,1000,금융투자,2018-10-01
2,009150,삼성전기,65446,28734,36712,9083102000,3996953000,5086149000,전자부품 제조업,1000,금융투자,2018-10-01
3,278540,KODEX MSCI Korea TR,500010,6,500004,4568666000,54780,4568611220,-,1000,금융투자,2018-10-01
4,263700,케어랩스,130000,1167,128833,3796260000,35978450,3760281550,소프트웨어 개발 및 공급업,1000,금융투자,2018-10-01
5,114800,KODEX 인버스,713264,222167,491097,4576877280,1422628845,3154248435,-,1000,금융투자,2018-10-01
6,000660,SK하이닉스,147663,110026,37637,10897341300,8125357800,2771983500,반도체 제조업,1000,금융투자,2018-10-01
7,032640,LG유플러스,240832,95293,145539,4192080950,1667713100,2524367850,전기 통신업,1000,금융투자,2018-10-01
8,223190,KODEX 200가치저변동,219802,101,219701,1909054490,879665,1908174825,-,1000,금융투자,2018-10-01
9,252730,KBSTAR 모멘텀로우볼,153961,30,153931,1854263970,361850,1853902120,-,1000,금융투자,2018-10-01


# CSV로 저장

일자별, 압축 CSV 파일 (예: investors_2018.csv.gz )

In [ ]:
import os

# 디렉토리가 없으면 생성
data_dir = './investors_data'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

# start ~ end 날짜까지 csv로 저장
start = datetime(2018, 1, 1)
end = datetime(2018, 12, 31)

for date in pd.date_range(start=start, end=end):
    fn = data_dir + '/' + 'investors_' + date.strftime('%Y%m%d') + '.csv.gz'
    if os.path.isfile(fn): # 파일이 존재하면 SKIP
        continue
    df = krx_investors(date)
    df.to_csv(fn, index=False, compression='gzip')
    print(date.date(), 'rows: ', len(df))

# 개별 CSV 파일을 DB 로 구축

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import re
import glob

create_table_query = """
CREATE TABLE IF NOT EXISTS investors (
  "날짜" TIMESTAMP,
  "종목코드" TEXT,
  "종목명" TEXT,
  "매수거래량" REAL,
  "매도거래량" REAL,
  "순매수거래량" REAL,
  "매수거래대금" REAL,
  "매도거래대금" REAL,
  "순매수거래대금" REAL,
  "업종명" TEXT,
  "투자자코드" TEXT,
  "투자자명" TEXT,
  UNIQUE ("날짜", "종목코드", "투자자코드") ON CONFLICT REPLACE
);

CREATE INDEX IF NOT EXISTS "ix_investors_날짜" ON "investors" ("날짜");
CREATE INDEX IF NOT EXISTS "ix_investors_종목코드" ON "investors" ("종목코드");
CREATE INDEX IF NOT EXISTS "ix_investors_투자자코드" ON "investors" ("투자자코드");
"""

db_fname = 'krx_investors.db'
con = sqlite3.connect(db_fname)
con.executescript(create_table_query)

data_dir = './investors_data'
csv_list = sorted(glob.glob(data_dir + '/investors_2018*'))

# 모든 파일명에 대해
for csv in csv_list:
    # 파일명에서 날짜 추출
    dt = pd.to_datetime(re.match('.*_(\d+)\.csv.gz', csv)[1], format='%Y%m%d')
    # DB에 해당 날짜 데이터가 있는지 조회
    query = 'select * from investors where 날짜=datetime("%s")' % dt.strftime('%Y-%m-%d')
    query_df = pd.read_sql(query, con, parse_dates=['날짜'])
    # DB에 없으면 추가
    if len(query_df) == 0:
        df = pd.read_csv(csv, parse_dates=['날짜'])        
        df.to_sql('investors', con, if_exists='append', index=False)
        print(dt.strftime('%Y-%m-%d'), len(df), 'rows')
    else:
        print(dt.strftime('%Y-%m-%d'), 'SKIP')


#### 2018 [FinanceData](http://financedata.kr)